In [8]:
!pip install --upgrade pip
!pip install groq gradio

In [9]:
# Restart the runtime after installation
# Go to: Runtime > Restart runtime

In [10]:
!pip list | grep groq

groq                                     1.0.0


In [ ]:
# Install required packages first
import subprocess
import sys

try:
    import groq
except ImportError:
    print("Installing required packages...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "groq"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "gradio"])
    print("Installation complete!")

import gradio as gr
from groq import Groq
import json

def is_geographic_question(question, api_key):
    """Check if the question is related to geography using Groq API"""
    try:
        client = Groq(api_key=api_key)

        system_prompt = """You are a classifier that determines if a question is related to geography.
Geography includes: countries, cities, capitals, mountains, rivers, oceans, continents, climate,
population, borders, landmarks, maps, coordinates, regions, terrain, and geographical features.

Respond with ONLY a JSON object in this exact format:
{"is_geographic": true} or {"is_geographic": false}

Do not include any other text or explanation."""

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Is this question related to geography? Question: {question}"}
            ],
            temperature=0.1,
            max_tokens=50
        )

        result = response.choices[0].message.content.strip()
        parsed = json.loads(result)
        return parsed.get("is_geographic", False)

    except Exception as e:
        return False

def get_geographic_response(question, api_key):
    """Get response for geographic questions using Groq API"""
    try:
        client = Groq(api_key=api_key)

        system_prompt = """You are a knowledgeable geography expert. Answer questions about:
- Countries, cities, capitals, and regions
- Physical geography (mountains, rivers, oceans, climate)
- Human geography (population, culture, languages)
- Maps, coordinates, and locations
- Landmarks and natural features

Provide accurate, informative, and concise answers."""

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
            temperature=0.7,
            max_tokens=1024
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"Error: {str(e)}"

def chat_interface(message, history, api_key):
    """Main chat interface function"""
    if not api_key or not api_key.strip():
        return "⚠️ Please enter your Groq API key to start chatting."

    if not message or not message.strip():
        return "Please enter a question."

    # Check if question is geographic
    if not is_geographic_question(message, api_key):
        return "🌍 I'm a geography specialist! I can only answer questions related to geography, such as countries, cities, mountains, rivers, climate, population, and geographical features. Please ask me a geography-related question!"

    # Get response for geographic question
    response = get_geographic_response(message, api_key)
    return response

# Create Gradio interface
with gr.Blocks(title="Geographic Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🌍 Geographic Chatbot
        Ask me anything about geography! I specialize in countries, cities, physical features,
        climate, population, and more.

        **Note:** I only answer geography-related questions.
        """
    )

    with gr.Row():
        api_key_input = gr.Textbox(
            label="Groq API Key",
            placeholder="Enter your Groq API key here...",
            type="password",
            scale=4
        )

    chatbot = gr.Chatbot(
        label="Chat",
        height=400,
        show_label=True
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Your Question",
            placeholder="Ask a geography question... (e.g., What is the capital of France?)",
            scale=4
        )
        submit_btn = gr.Button("Send", scale=1, variant="primary")

    gr.Examples(
        examples=[
            "What is the highest mountain in the world?",
            "Tell me about the Amazon River",
            "What is the capital of Japan?",
            "Which countries border Germany?",
            "What is the population of India?"
        ],
        inputs=msg
    )

    def respond(message, chat_history, api_key):
        if not message.strip():
            return chat_history, ""

        bot_message = chat_interface(message, chat_history, api_key)
        chat_history.append((message, bot_message))
        return chat_history, ""

    msg.submit(respond, [msg, chatbot, api_key_input], [chatbot, msg])
    submit_btn.click(respond, [msg, chatbot, api_key_input], [chatbot, msg])

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True)

/tmp/ipython-input-1802374672.py:94: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Geographic Chatbot", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1802374672.py:113: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1802374672.py:113: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://357c95393ef6275609.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
